In [142]:
# imports
import pandas as pd
import numpy as np
import xlrd as xl
import matplotlib.pyplot as plt
... 

Ellipsis

In [143]:
#initialize main dataframe 
#change the main names most likely
#don't forget to write to an excel file (pd manual)

df = pd.DataFrame(columns=['Filename', 'Signal', 'RT [min]', 'Type', 'Width [min]', 'Area', 'Height', 'Peak Symmetry'])
df

,Filename,Signal,RT [min],Type,Width [min],Area,Height,Peak Symmetry


In [144]:
#initialize main dataframe 
#df = pd.DataFrame()

#initialize a tester dataframe
tester = pd.read_excel(r"C:\Users\aniko\repo\auto-lab\data\tester.XLS", sheet_name="Sheet1")
tester

,Area Percent Report,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Data file:,NaN,NaN,NaN,NaN,NaN,C:\Chem32\1\Data\2022-03-10_t44_L1-15_triplica...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sample name:,NaN,NaN,NaN,NaN,NaN,A1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Description:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Sample amount:,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,Sample type:,NaN,NaN,NaN,NaN,Sample,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Instrument:,NaN,NaN,NaN,NaN,NaN,LCMS 1,NaN,NaN,NaN,...,NaN,NaN,NaN,Location:,NaN,NaN,NaN,NaN,P1-A1,NaN
8,Injection date:,NaN,NaN,NaN,NaN,NaN,3/10/2022 9:36:10 AM,NaN,NaN,NaN,...,NaN,NaN,NaN,Injection:,NaN,NaN,NaN,NaN,1 of 1,NaN
9,Acq. method:,NaN,NaN,NaN,NaN,NaN,2021-12-14.MUC.J.M,NaN,NaN,NaN,...,NaN,NaN,NaN,Injection volume:,NaN,NaN,NaN,NaN,2.000,NaN


In [145]:
def find_file():
       datafile = ""
       #using exceptions to break out of a double for loop :3
       class FileFound(Exception): pass
       try:
              for r in range (tester.shape[0]):
                     for c in range (tester.shape[1]):
                            if(tester.iat[r,c] == "Data file:"):
                                   raise FileFound
       except FileFound:
              datafile = tester.iat[r, c + 6]
       
       return datafile

In [146]:

def sig_iterator(row, col, filename):
    sig_name = tester.iat[row, col + 2]
    datarow = []
    
    #loops until end of signal information
    class SingleSig(Exception): pass

    try:
        for r in range (row + 2, tester.shape[0]):
            datarow = [filename, sig_name]
            for c in range (col, tester.shape[1]):
                if(tester.iat[r,c] == "Sum"):
                    raise SingleSig
                if(pd.notnull(tester.iat[r,c])):
                    datarow.append(tester.iat[r,c])

            #append row to the dataframe        
            df.loc[len(df)] = datarow

    except SingleSig:
        ...

In [147]:
#main reader method: xls reader
#may need input as xls file
def xls_reader():
    #finds filename:
    filename = find_file()
    #should I have it looking for "Signal":? should be in helper actually. maybe just a general iterator
    
    for r in range (tester.shape[0]):
        for c in range (tester.shape[1]):
            if(tester.iat[r, c] == "Signal:"):
                sig_iterator(r, c, filename)


xls_reader()

In [148]:
df

,Filename,Signal,RT [min],Type,Width [min],Area,Height,Peak Symmetry
0,C:\Chem32\1\Data\2022-03-10_t44_L1-15_triplica...,"MSD1 TIC, MS File",0.150,BBA,0.0880,1709251.3750,300407.0313,0.63424
1,C:\Chem32\1\Data\2022-03-10_t44_L1-15_triplica...,"MSD1 TIC, MS File",0.150,BBA,0.0880,1709251.3750,300407.0313,0.63424
2,C:\Chem32\1\Data\2022-03-10_t44_L1-15_triplica...,"MSD2 TIC, MS File",0.149,BBA,0.0857,46094.8164,8393.1582,0.61030
3,C:\Chem32\1\Data\2022-03-10_t44_L1-15_triplica...,"VWD1 A, Wavelength=257 nm",0.082,BBA,0.0502,901.7473,283.4999,0.77009


In [149]:
'''
def to_xls(df):
    with pd.ExcelWriter("path_to_file.xlsx") as writer:  
        df.to_excel(writer)


to_xls(df)
'''


'\ndef to_xls(df):\n    with pd.ExcelWriter("path_to_file.xlsx") as writer:  \n        df.to_excel(writer)\n\n\nto_xls(df)\n'